# Data preparation – interdependency networks

Prepared by Omar A. Guerrero (oguerrero@turing.ac.uk, @guerrero_oa)

In the literature related to the Sustainable Development GOals (SDGs), much attention has been given to interdependency networks between SDGs, targets, or development indicators. On of the features of PPI is its ability to take into account such networks as an exogenous variable meant to preserve certain structure in the co-movenent of the indicators. This network is considered exogenous because it is a stylised fact of the system under study, not a causal account of the relationship between the indicators. While many studies attempt at making causal claims from such objects, we have shown (in the book and in multiple publications) that such statements cannot be causal (see https://doi.org/10.1016/j.im.2020.103342 for an example). Thus, the aim in this tutorial is to simply show how to prepare the data for the network input of PPI.

In the book, as in most of PPI's studies, we have employed a method called `sparsebn` (see http://doi.org/10.18637/jss.v091.i11). However, for the sake of simplicity in these tutorials, let us employ a simple correlation approach to construct the network. First, we will lead the clean indicator data. Then, we will estimate pairwise correlations. Next, we will filter out edges using an arbitrary threshold criterion. Finally, we will structure the data and export it.

## Import the necessary python libraries to manipulate data

In [1]:
import pandas as pd
import numpy as np